<a href="https://colab.research.google.com/github/Jeevikavishwakarma/PRODIGY_DS_04/blob/main/Internship_task_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [109]:
#importing
!pip install kaggle

In [110]:
#Configuring kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [111]:
#Importing the dataset
!kaggle datasets download -d kazanova/sentiment140    # -d owner/filename

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 90% 73.0M/80.9M [00:00<00:00, 161MB/s]
100% 80.9M/80.9M [00:00<00:00, 143MB/s]


In [112]:
# Extracting the zip file
from zipfile import ZipFile
dataset = "/content/sentiment140.zip"
with ZipFile(dataset, "r") as zip:
  zip.extractall()
  print("The dataset is extracted")


The dataset is extracted


In [113]:
#Importing the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import ConfusionMatrixDisplay

In [114]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [115]:
#Data Processing
df = pd.read_csv("/content/training.1600000.processed.noemoticon.csv", encoding = 'ISO-8859-1')

In [116]:
df.shape

(1599999, 6)

In [117]:
df.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [118]:
#As the first value is been read as a column header
columns_names = ['target','id','date','flag','user','text']
df = pd.read_csv("/content/training.1600000.processed.noemoticon.csv", names = columns_names, encoding ='ISO-8859-1')

In [119]:
df.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [120]:
#Missing values
df.isnull().sum()
df.shape

(1600000, 6)

In [121]:
df['target'].value_counts()
#Here the distribution is equal
#If the distribution is not equal then we have to do upsampling and downsampling

,count
target,
0,800000
4,800000


In [122]:
#Label converting
df.replace({'target':{4:1}}, inplace =True)

In [123]:
import pandas as pd
from sklearn.utils import resample

# Separate classes
df_majority = df[df['target'].values == 1]
df_minority = df[df['target'].values == 0]

# Downsample majority class
df_majority_downsampled = resample(df_majority,replace=False,n_samples= 5000, random_state=42)
df_minority_downsampled = resample(df_minority,replace=False,n_samples =5000, random_state=42)
# Combine balanced dataset
df = pd.concat([df_majority_downsampled, df_minority_downsampled])
print(df.value_counts())  # Balanced dataset


target  id          date                          flag      user            text                                                                                                                                      
0       1467855673  Mon Apr 06 22:31:29 PDT 2009  NO_QUERY  IluvJoelMadden  Going to school soon... can't find anything to wear!! gosh it's so hard                                                                       1
1       1966800464  Fri May 29 18:25:25 PDT 2009  NO_QUERY  JT_Gunner       Had fun hanging out with some old childhood friends and celebrating his dad's b-day.                                                          1
        1966374387  Fri May 29 17:39:00 PDT 2009  NO_QUERY  reeshpeesh16    mood: happy. the other day was like the best day of my life.  [insider]  sonq: bumpinq to &quot;i need a qirl&quot; -trey sonqs.              1
        1966375179  Fri May 29 17:39:05 PDT 2009  NO_QUERY  kaa93           @tommcfly http://twitpic.com/47rhq - i love yours

In [124]:
df['target'].value_counts()

,count
target,
1,5000
0,5000


In [139]:
#Stemming
port = PorterStemmer()
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]',' ',content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

In [140]:
df['stemmed_content'] = df['text'].apply(stemming)

In [141]:
df.head()

,target,id,date,flag,user,text,stemmed_content
1012188,1,1881179620,Fri May 22 03:51:54 PDT 2009,NO_QUERY,tarawade,Is lookin 4ward to a long weekend really dont...,lookin ward long weekend realli dont want go w...
1099036,1,1970537555,Sat May 30 04:15:49 PDT 2009,NO_QUERY,Millie_stillie,#myweakness Is music and i live to meet the p...,myweak music live meet peopl make
1275978,1,2001154935,Tue Jun 02 00:00:21 PDT 2009,NO_QUERY,zsangel,figured out the Internet on my new iPod,figur internet new ipod
1388988,1,2053074174,Sat Jun 06 03:10:08 PDT 2009,NO_QUERY,krisignacio,@hillsongunited can't wait to worship with you...,hillsongunit wait worship guy tonight much fun
938859,1,1793548492,Thu May 14 03:06:55 PDT 2009,NO_QUERY,_DrInE_,@sillybeggar Congrats James !! I'm sure the bo...,sillybeggar congrat jame sure book go huge suc...


In [142]:
#Separating the data and label
X = df['stemmed_content'].values
Y = df['target'].values

In [144]:
#Splitting the data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size =0.2, random_state = 2, stratify = Y)
print(X_train)

['came back run park' 'go bed alon first time week'
 'love jane austen read book whenev free time' ...
 'ps share great littl set want get crowd' 'fuck chi straightner broken'
 'quot hey look great quot way']


In [145]:
#Feature Extraction
#Converting text to numerical format
# X_train  = str(X_train)  # Remove this line
# X_test = str(X_test)    # Remove this line
vector = TfidfVectorizer()
# Join the stemmed words back into sentences for each document

X_train = vector.fit_transform(X_train)  # Use the list of sentences
X_test = vector.transform(X_test)      # Use the list of sentences

In [146]:
#Training the model
#Logistic Regression
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=1000)
model.fit(X_train,Y_train)

LogisticRegression(max_iter=1000)

In [147]:
#Model Evaluation
#Accuracy score of the model
X_train_predict = model.predict(X_train)
accuracy_train = accuracy_score(Y_train, X_train_predict)
print("Accuracy score of training data:")

Accuracy score of training data:


In [148]:
accuracy_test = accuracy_score(Y_test, model.predict(X_test))
print("Accuracy score of testing data:")

Accuracy score of testing data:


In [149]:
#Saving the Trained model
import pickle
filename = 'Sentiment_Analysis_model.sav'
pickle.dump(model,open(filename,'wb'))

In [152]:
#Using the saved model for future predictions
load_model = pickle.load(open('Sentiment_Analysis_model.sav','rb'))   #file path

In [153]:
X_new = X_test[200]
print(Y_test[200])

pred = model.predict(X_new)
print(pred)

if pred[0] == 0:
  print("Negative Tweet")

else:
  print("Positive Tweet")

1
[1]
Positive Tweet


In [154]:
X_new = X_test[20]
print(Y_test[20])

pred = model.predict(X_new)
print(pred)

if pred[0] == 0:
  print("Negative Tweet")

else:
  print("Positive Tweet")

0
[0]
Negative Tweet
